# axialize scans

### import packages and set up

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
import pickle
from distutils.spawn import find_executable
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)

In [2]:
analysis_version = "2018_07_03"
project_dir_absolute = Path('~/Downloads/euler_number_prediction').expanduser()
%pwd
%cd {project_dir_absolute}
%pwd

/Users/rodgersleejg/Downloads/euler_number_prediction


'/Users/rodgersleejg/Downloads/euler_number_prediction'

In [3]:
Path(find_executable('afni'))

PosixPath('/Users/rodgersleejg/abin/afni')

In [4]:
orig_scans  = project_dir_absolute / 'orig_scans'
output_dir = project_dir_absolute / 'axialized_scans'
afni_dir = Path('~').expanduser() / 'abin'
if not afni_dir.exists():

        afni_path = Path(find_executable('afni'))
        if not afni_path.exists():
            raise EnvironmentError
        else: 
            afni_dir = afni_path.parent

### Create template

In [5]:
template_dir = project_dir_absolute / 'template_dir'
if not template_dir.exists():
    template_dir.mkdir()
template_glob = 'MNI152_T1_2009c+tlrc.*'
input_template = template_dir / 'MNI152_T1_2009c+tlrc.BRIK'
mask = template_dir / 'output_mask+tlrc.BRIK.gz'

%cd {template_dir}
!rm {mask} {sub_weight_template} {mask} ELL.nii ELLBLUR.nii output_mask*
! cp {afni_dir /template_glob} {template_dir};cd {template_dir}; gunzip -f {template_glob + 'gz'}

# Generate template brainmask
# !3dAutomask -dilate 1 -prefix {mask.name.split('+')[0]} {input_template.name}

# Generate ellipsoid
eli_cmd = f"3dcalc -a {input_template} -expr 'step((1-(x-0)*(x-0)/(30**2)-(y-9)*(y-9)/(50**2)-(z-2)*(z-2)/(30**2)))' -prefix ELL.nii -float"
!{eli_cmd}
!3dmerge -1blur_rms 4.2 -prefix ELLBLUR.nii ELL.nii

# Make sub-cortical weighted template
sub_weight_template = input_template.as_posix().replace('+tlrc','_wtell+tlrc')
!3dcalc -a {input_template} -b ELLBLUR.nii -expr 'a*(1+5*b)' -prefix {sub_weight_template}

/Users/rodgersleejg/Downloads/euler_number_prediction/template_dir
rm: {mask}: No such file or directory
rm: {sub_weight_template}: No such file or directory
rm: {mask}: No such file or directory
rm: ELL.nii: No such file or directory
rm: ELLBLUR.nii: No such file or directory
rm: output_mask*: No such file or directory
++ 3dcalc: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
++ Authored by: A cast of thousands
*+ WARNING: input 'a' is not used in the expression
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./ELL.nii
Program 3dmerge 
++ 3dmerge: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
3dmerge: edit and combine 3D datasets, by RW Cox
++ default -1dindex = 0
++ default -1tindex = 1
++ editing input dataset in memory (32.5 MB)
.
-- Wrote edited dataset: ./ELLBLUR.nii
++ 3dcalc: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset /Users/rodgersleejg/Do

### Axialize example

In [6]:
scans = pd.DataFrame({'scan_path':[p.as_posix() for p in orig_scans.glob('*.nii.gz')]})

In [7]:
scans.head()


,scan_path
0,/Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz
1,/Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-0414_ses-1_acq-mprage_run-002_T1w.nii.gz
2,/Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-1223_ses-1_acq-mprage_run-001_T1w.nii.gz
3,/Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-1360_ses-1_acq-mprage_run-002_T1w.nii.gz
4,/Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-1491_ses-1_acq-mprage_run-002_T1w.nii.gz


In [8]:
scan_path = scans.scan_path.iloc[0]

scan_path

'/Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz'

In [9]:
def make_fatproc_cmd(scan_path,input_template,output_dir=None,sub_weight_template = None, project_dir_absolute = Path.cwd()):
    if not output_dir:
        output_dir = project_dir_absolute / 'axialized_scans'
    if not output_dir.exists():
        output_dir.mkdir()
        
    cmd = f"""\
    fat_proc_axialize_anat\
        -inset  {scan_path}\
        -prefix {output_dir / Path(scan_path).name.split('.')[0]}\
        -mode_t1w\
        -refset          {input_template}\
        -out_match_ref \
        """
    
    if sub_weight_template:
        cmd += f' -extra_al_wtmask {sub_weight_template}'
    return cmd

In [10]:
cmd = make_fatproc_cmd(scan_path,input_template,sub_weight_template=sub_weight_template)

In [11]:
!{cmd}

++ Start script version: 4.1
++ 3dinfo: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
118
++ 3dinfo: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz to FLOAT32
     Warnings of this type will be muted for this session.
     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
++ Found input file:   /Users/rodgersleejg/Downloads/euler_number_prediction/orig_scans/sub-1289_ses-1_acq-mprage_run-001_T1w.nii.gz
++ 3dinfo: AFNI version=AFNI_18.2.04 (Jul  6 2018) [64-bit]
++ Found input file:   /Users/rodgersleejg/Downloads/e

### Axialize using Dask

In [167]:
from dask.distributed import Client, LocalCluster
from dask import delayed
import subprocess

In [139]:
# cluster = LocalCluster(n_workers=1,threads_per_worker=2)
client = Client(processes = False,
    n_workers= 1,
    threads_per_worker=2
    )

In [170]:
@delayed
def run_cmd(cmd):
    pp = subprocess.run(cmd,shell=True,stdout=subprocess.PIPE,stderr= subprocess.PIPE)
    return pp

    

In [171]:
cmds = [make_fatproc_cmd(x,input_template,output_dir,sub_weight_template,project_dir_absolute) for x in scans.scan_path]
graph = [run_cmd(cmd) for cmd in cmds]

In [172]:
result = client.compute(graph)

In [187]:
outputs = client.gather(result)

### Inspecting output

In [193]:
pickle.dump( outputs, open( output_dir / 'axialize_output.pklz', "wb" ) )

In [194]:
# outputs_recovered = pickle.load( open( output_dir / 'axialize_output.pklz', "rb" ) )

In [182]:
result[0]


<Future: status: finished, type: CompletedProcess, key: run_cmd-44cfaaa4-95a0-409f-b8ee-a2e38f6d5a9b>

In [177]:
client

Client Scheduler: inproc://10.0.0.9/36627/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 2 Memory: 8.59 GB


In [184]:
%ls {output_dir}|head

QC/
output_not_required+orig.BRIK
output_not_required+orig.HEAD
sub-0102_ses-1_acq-mprage_run-001_T1w.nii.gz
sub-0102_ses-1_acq-mprage_run-001_T1w_12dof.param.1D
sub-0102_ses-1_acq-mprage_run-001_T1w_cmd.txt
sub-0150_ses-1_acq-mprage_run-001_T1w.nii.gz
sub-0150_ses-1_acq-mprage_run-001_T1w_12dof.param.1D
sub-0150_ses-1_acq-mprage_run-001_T1w_cmd.txt
sub-0150_ses-1_acq-mprage_run-002_T1w.nii.gz
